In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import shutil
from google.colab import files

Mounted at /content/drive/


In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense, Flatten, LSTM, Dropout, SimpleRNN, Lambda, BatchNormalization
from keras.models import Sequential
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical


In [3]:
data_path = "/content/drive/MyDrive/Location_Season_Data/"

In [4]:
df = pd.read_csv(data_path + "Location_Season_Wind_Pressure_first_3_year_ROI.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,448,449,450,451,452,453,season,Name,Day,Class
0,0.475692,0.459692,0.473846,0.454769,0.471385,0.449846,0.468923,0.444923,0.465846,0.440000,...,0.448598,0.376324,1,1,1,0,2,116_c7_cp2_100_32,116,0
1,0.464615,0.488000,0.463385,0.491692,0.462769,0.494769,0.460923,0.498462,0.460308,0.502769,...,0.480997,0.514019,1,0,1,0,2,125_c3_cp2_49_32,125,0
2,0.248615,0.569231,0.243692,0.564923,0.240000,0.560615,0.237538,0.556923,0.235692,0.553231,...,0.267912,0.554517,1,0,0,0,2,120_c1_cp2_31_29,120,0
3,0.552000,0.515077,0.544000,0.517538,0.536615,0.519385,0.529846,0.521846,0.524923,0.523692,...,0.499065,0.442368,1,1,1,0,2,127_c3_cp3_96_38,127,0
4,0.248615,0.552000,0.243692,0.544000,0.240000,0.536615,0.237538,0.529846,0.235692,0.524923,...,0.267912,0.499065,1,0,0,0,2,120_c2_cp2_31_29,120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,0.930833,0.396694,0.935248,0.409836,0.939601,0.426230,0.937135,0.433333,0.942242,0.434426,...,0.953185,0.365079,1,0,1,1,3,997_c12_cp2_62_59,997,3
1064,0.869459,0.438017,0.873418,0.459016,0.876279,0.475410,0.877193,0.483333,0.879589,0.483607,...,0.919884,0.682540,1,0,1,1,3,998_c12_cp2_37_48,998,2
1065,0.759864,0.404959,0.772639,0.418033,0.783731,0.434426,0.791910,0.433333,0.800783,0.418033,...,0.813224,0.142857,1,1,1,0,3,999_c11_cp2_102_34,999,2
1066,0.812957,0.338843,0.820837,0.336066,0.826595,0.319672,0.829435,0.300000,0.835047,0.278689,...,0.921332,0.706349,1,0,1,0,3,999_c11_cp3_37_44,999,2


In [5]:
df_train = df[(df["Day"] >= 0) & (df["Day"] <= 730)]
X_train = df_train.drop(columns = ["450", "451", "452", "453", "season", "Name", "Day", "Class"])
Y_train = df_train["Class"]

df_test = df[(df["Day"] >= 731) & (df["Day"] <= 1095)]
X_test = df_test.drop(columns = ["450", "451", "452", "453", "season", "Name", "Day", "Class"])
Y_test = df_test["Class"]

In [6]:
#Reshape training and testing data
train_X = X_train.to_numpy().reshape(X_train.shape[0],15,15,2).astype("float32")

train_Y = Y_train.to_numpy()

test_X = X_test.to_numpy().reshape(X_test.shape[0],15,15,2).astype("float32")

test_Y = Y_test.to_numpy()

train_Y = to_categorical(train_Y, num_classes=4)
test_Y = to_categorical(test_Y, num_classes=4)

In [16]:
#Define the model architecture
model_rnn = Sequential()
model_rnn.add(Lambda(lambda x: x[:,:,:,0], input_shape = (15,15,2)))
model_rnn.add(LSTM(16, return_sequences=True))
model_rnn.add(BatchNormalization())
model_rnn.add(Dropout(0.1))
model_rnn.add(LSTM(16, return_sequences=True))
model_rnn.add(BatchNormalization())
model_rnn.add(Dropout(0.1))
model_rnn.add(LSTM(16))
model_rnn.add(BatchNormalization())
model_rnn.add(Dense(4, activation='softmax'))

In [17]:
model_rnn.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [18]:
early_stopping = EarlyStopping(monitor='loss', patience=30)


#Fit the model
model_rnn.fit(train_X, train_Y, epochs = 1000, batch_size = 64, verbose = 2, callbacks = early_stopping)

Epoch 1/1000
12/12 - 5s - loss: 1.1297 - accuracy: 0.5550 - 5s/epoch - 431ms/step
Epoch 2/1000
12/12 - 0s - loss: 0.7719 - accuracy: 0.7381 - 169ms/epoch - 14ms/step
Epoch 3/1000
12/12 - 0s - loss: 0.6828 - accuracy: 0.7585 - 179ms/epoch - 15ms/step
Epoch 4/1000
12/12 - 0s - loss: 0.6278 - accuracy: 0.7870 - 165ms/epoch - 14ms/step
Epoch 5/1000
12/12 - 0s - loss: 0.5764 - accuracy: 0.8046 - 167ms/epoch - 14ms/step
Epoch 6/1000
12/12 - 0s - loss: 0.5932 - accuracy: 0.7748 - 167ms/epoch - 14ms/step
Epoch 7/1000
12/12 - 0s - loss: 0.5440 - accuracy: 0.8141 - 178ms/epoch - 15ms/step
Epoch 8/1000
12/12 - 0s - loss: 0.5234 - accuracy: 0.8005 - 165ms/epoch - 14ms/step
Epoch 9/1000
12/12 - 0s - loss: 0.4945 - accuracy: 0.8195 - 169ms/epoch - 14ms/step
Epoch 10/1000
12/12 - 0s - loss: 0.4953 - accuracy: 0.8277 - 167ms/epoch - 14ms/step
Epoch 11/1000
12/12 - 0s - loss: 0.5047 - accuracy: 0.7951 - 171ms/epoch - 14ms/step
Epoch 12/1000
12/12 - 0s - loss: 0.4563 - accuracy: 0.8290 - 163ms/epoch - 1

In [19]:
# predict class probabilities for test set
Y_pred = model_rnn.predict(test_X)

# convert predicted probabilities to class labels
Y_pred_labels = np.argmax(Y_pred, axis=1)


# convert true labels to class labels
Y_true_labels = np.argmax(test_Y, axis=1)

#Accuracy
accuracy = accuracy_score(Y_true_labels, Y_pred_labels)
print(accuracy)

# compute confusion matrix
conf_matrix = confusion_matrix(Y_true_labels, Y_pred_labels)

print(conf_matrix)

11/11 [==============================] - 1s 3ms/step
0.5377643504531722
[[66  2 21  0]
 [ 1 34  3 41]
 [11  0 76  0]
 [ 1  0 73  2]]
